In [1]:
%matplotlib inline
!pip install nltk
!pip install seaborn
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
con = sqlite3.connect('database.sqlite') 

In [2]:
selected_data=pd.read_sql("select * from reviews",con)

In [3]:
selected_data.shape

(568454, 10)

In [4]:
selected_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
### Trying out some queries..
number_of_unique_users=pd.value_counts(selected_data['UserId'])
print("Total_no_of_unique_users {}".format(len(number_of_unique_users)))

Total_no_of_unique_users 256059


In [6]:
filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""", con) 

In [7]:
filtered_data.shape

(525814, 10)

In [8]:
def conversion_from_review_to_string(review):
    if review==4 or review==5:
        return 'positive'
    else:
        return 'negative'

In [9]:
## Trying to remove score column and replace it by +/- string..
filtered_data['Score']=filtered_data['Score'].map(conversion_from_review_to_string)

In [10]:
len(filtered_data['Score'])

525814

In [11]:
filtered_data['Score'].value_counts()

positive    443777
negative     82037
Name: Score, dtype: int64

In [12]:
filtered_data.shape

(525814, 10)

In [13]:
filtered_data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [14]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [15]:
filtered_data=filtered_data.sort_values('ProductId',axis=0)
past_data_size=filtered_data.shape[0]
past_data_size

525814

In [16]:
filtered_data.drop_duplicates({'ProductId','UserId','ProfileName','Text'},inplace=True)
print(filtered_data.shape)
new_data_shape=filtered_data.shape[0]

(524587, 10)


In [17]:
data_loss=(past_data_size-new_data_shape)
print("Number of such duplicates are {}".format(data_loss))

Number of such duplicates are 1227


In [18]:
filtered_data=filtered_data[filtered_data.HelpfulnessNumerator<=filtered_data.HelpfulnessDenominator]

In [19]:
### So we have such duplicates this will affect the performance of the classifier
### Calculation the number of positive and negative reviews..
filtered_data['Score'].value_counts()

positive    442867
negative     81718
Name: Score, dtype: int64

In [20]:
filtered_data['Text'][10]

"I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!"

In [21]:
import re
def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

### To apply LSTM on Amazon fine food reviews the approach i am going to take is:
#### 1. find the vocabolary i.e all possible set of words in the dataset.
#### 2. find the word,freq of each word in the data.
#### 3. Sort the data on descending order of frequency.

In [22]:
score=filtered_data['Score']

In [23]:
score.value_counts()

positive    442867
negative     81718
Name: Score, dtype: int64

In [24]:
cleaner_text=[]
for sent in filtered_data['Text'].values:
    small_filter=[]
    small_sent=sent
    small_sent=cleanhtml(small_sent)
    small_sent=cleanpunc(small_sent)
    for each_word in small_sent.split():
        if(len(each_word)>2 and each_word.isalpha()):
            small_filter.append(each_word)
        else:
            continue
    cleaner_text.append(" ".join(small_filter))

In [25]:
filtered_data['goodText']=cleaner_text

In [26]:
cleaner_text[200]

'You asked for review this purchase already said fast shipping and good product not bother with these reviews any more have problem will let you know will stop using amozon the future you dont leave alone'

In [27]:
del cleaner_text

In [28]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,goodText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,this witty little book makes son laugh loud re...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew reading these Sendak books and watching t...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,This fun way for children learn their months t...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,This great little book read has nice rhythm we...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,This book poetry about the months the year goe...


In [29]:
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
filtered_data.to_sql('Reviews', conn,  schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

In [30]:
vocab_set=set()
for each_sentence in filtered_data['goodText']:
    small_list=each_sentence.split()
    for each_word in small_list:
        vocab_set.add(each_word.lower())

In [31]:
count_freq={}
for each_sentence in filtered_data['goodText']:
    for each_word in each_sentence.split():
        if(each_word.lower() in vocab_set):
            if(count_freq.get(each_word.lower()) is None):
                count_freq[each_word.lower()]=1
            else:
                count_freq[each_word.lower()]=count_freq[each_word.lower()]+1

In [32]:
new_dict=[(v,k) for k,v in count_freq.items()]

In [33]:
new_dict=sorted(new_dict,reverse=True)

In [34]:
new_dict=[(t[1],t[0]) for t in new_dict]

In [35]:
new_dict=dict(new_dict)

In [36]:
del count_freq

In [37]:
## Now i got the sorted dictionary..
top_words=5000
data_lst=[]
words_list=list(new_dict.keys())

In [38]:
for each_sentence in filtered_data['goodText']:
    small_lst=[]
    word_list=each_sentence.split()
    for each_word in word_list:
        if(words_list.index(each_word.lower())>5000):
            continue
        else:
            small_lst.append(words_list.index(each_word.lower()))
    data_lst.append(small_lst)

In [39]:
max_len=-10
for i in range(len(data_lst)):
    if(len(data_lst[i])>max_len):
        max_len=len(data_lst[i])
print(max_len)

1632


In [40]:
data_lst=np.array(data_lst)

In [41]:
data_lst.shape

(524585,)

In [42]:
new_list=data_lst

In [43]:
new_list.shape

(524585,)

In [44]:
new_list.shape

(524585,)

In [45]:
# Credits: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


In [46]:
np.random.seed(7)

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
def pos_neg_1_0(a):
    new_lst=[]
    for i in range(len(a)):
        if(a.iloc[i] == 'positive'):
            new_lst.append(1)
        else:
            new_lst.append(0)
    return new_lst

In [49]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,goodText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,this witty little book makes son laugh loud re...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew reading these Sendak books and watching t...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,This fun way for children learn their months t...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,This great little book read has nice rhythm we...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,This book poetry about the months the year goe...


In [50]:
filtered_data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text',
       'goodText'],
      dtype='object')

In [51]:
score=pos_neg_1_0(filtered_data['Score'])

In [52]:
X_train, X_test, y_train, y_test = train_test_split(new_list, score, random_state=42)

In [53]:
X_train.shape

(393438,)

In [54]:
max_review_length = 600
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print(X_train.shape)
print(X_train[1])

(393438, 600)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0   

In [55]:
X_train.shape

(393438, 600)

In [56]:
# create the model
from keras.initializers import glorot_normal
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(10))
model.add(Dense(1, kernel_initializer=glorot_normal(seed=None),activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#Refer: https://datascience.stackexchange.com/questions/10615/number-of-parameters-in-an-lstm-model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                1720      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 161,731
Trainable params: 161,731
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
model.fit(X_train, y_train, nb_epoch=4, batch_size=64,verbose=1,validation_data=(X_test,y_test))
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

C:\Users\vishnuvardhanreddy\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 393438 samples, validate on 131147 samples
Epoch 1/4
393438/393438 [==============================] - 7990s 20ms/step - loss: 0.1847 - acc: 0.9287 - val_loss: 0.1492 - val_acc: 0.9426
Epoch 2/4
393438/393438 [==============================] - 7501s 19ms/step - loss: 0.1316 - acc: 0.9508 - val_loss: 0.1333 - val_acc: 0.9502
Epoch 3/4
393438/393438 [==============================] - 7540s 19ms/step - loss: 0.1099 - acc: 0.9597 - val_loss: 0.1229 - val_acc: 0.9549
Epoch 4/4
131147/131147 [==============================] - 1736s 13ms/step
Accuracy: 95.78%


In [58]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 95.78%
